In [2]:
# Import necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, sys
# Add the 'scripts' directory to the Python path for module imports
sys.path.append(os.path.abspath(os.path.join('..', 'scripts')))

# Set max rows and columns to display
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)



In [3]:
# Configure logging
from logging_setup import SetupLogger
# Assuming this class is defined in scripts/
from data_preprocessor import DataPreprocessor  

logger = SetupLogger(log_file='../logs/notebooks.log').get_logger()

# Set figure size
plt.figure(figsize=(12, 5))

<Figure size 1200x500 with 0 Axes>

<Figure size 1200x500 with 0 Axes>

In [4]:
# Set google drive url for loading the dataset
data = 'https://drive.google.com/file/d/1Vj0JN3-cICvpks3nsEUtmtsY8eSWYW8M/view?usp=sharing'

# Setup the data preprocessor class
processor = DataPreprocessor(data, logger=logger)
# Load the data
price_data = processor.load_data()

Downloading...
From: https://drive.google.com/uc?export=download&id=1Vj0JN3-cICvpks3nsEUtmtsY8eSWYW8M
To: /home/abel/CODE/KiFiya/10-Academy-Week-10/data/data.csv
100%|██████████| 156k/156k [00:00<00:00, 352kB/s]
/home/abel/CODE/KiFiya/10-Academy-Week-10/scripts/data_preprocessor.py:51: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.data['Date'] = pd.to_datetime(self.data['Date'].str.strip(),  errors='coerce')


In [5]:
# Explore the first 10 rows
price_data.head(10)

,Date,Price
0,1987-05-20,18.63
1,1987-05-21,18.45
2,1987-05-22,18.55
3,1987-05-25,18.60
4,1987-05-26,18.63
5,1987-05-27,18.60
6,1987-05-28,18.60
7,1987-05-29,18.58
8,1987-06-01,18.65
9,1987-06-02,18.68


In [6]:
# Define the events dictionary
events_dict = {
    "Gulf War (1990-1991)": {"start": "1990-08-02", "end": "1991-02-28"},
    "September 11 Attacks (2001)": {"start": "2001-09-11", "end": "2001-09-11"},
    "2003 Invasion of Iraq": {"start": "2003-03-20", "end": "2011-12-18"},
    "2008 Financial Crisis": {"start": "2008-09-15", "end": "2009-07-01"},
    "Arab Spring (2010-2012)": {"start": "2010-12-18", "end": "2012-12-18"},
    "Libyan Civil War (2011)": {"start": "2011-02-15", "end": "2011-10-23"},
    "U.S. Shale Oil Boom (2010s)": {"start": "2010-01-01", "end": "2019-12-31"},
    "OPEC Production Cuts (2016-2017)": {"start": "2016-11-30", "end": "2017-06-30"},
    "Iran Nuclear Deal and Sanctions (2015-2018)": {"start": "2015-07-14", "end": "2018-05-07"},
    "COVID-19 Pandemic (2020)": {"start": "2020-03-11", "end": "2022-09-30"},
    "OPEC+ Response to COVID-19 (2020)": {"start": "2020-04-09", "end": "2020-12-31"},
    "Russia-Ukraine Conflict (2022)": {"start": "2022-02-24", "end": "2022-09-30"},
    "Global Economic Recovery (2021-2022)": {"start": "2021-01-01", "end": "2022-09-30"}
}

# Convert the events dictionary to a DataFrame
events_data = pd.DataFrame.from_dict(events_dict, orient='index')
events_data.reset_index(inplace=True)
events_data.columns = ['Event', 'Start', 'End']

# Convert Start and End to datetime
events_data['Start'] = pd.to_datetime(events_data['Start'])
events_data['End'] = pd.to_datetime(events_data['End'])

In [8]:
events_data

,Event,Start,End
0,Gulf War (1990-1991),1990-08-02,1991-02-28
1,September 11 Attacks (2001),2001-09-11,2001-09-11
2,2003 Invasion of Iraq,2003-03-20,2011-12-18
3,2008 Financial Crisis,2008-09-15,2009-07-01
4,Arab Spring (2010-2012),2010-12-18,2012-12-18
5,Libyan Civil War (2011),2011-02-15,2011-10-23
6,U.S. Shale Oil Boom (2010s),2010-01-01,2019-12-31
7,OPEC Production Cuts (2016-2017),2016-11-30,2017-06-30
8,Iran Nuclear Deal and Sanctions (2015-2018),2015-07-14,2018-05-07
9,COVID-19 Pandemic (2020),2020-03-11,2022-09-30


In [9]:
#Combine the price data with event data
def merge_event(events_data: pd.DataFrame, price_data: pd.DataFrame) -> pd.DataFrame:
    # Ensure Date, Start, and End columns are in datetime format
    price_data['Date'] = pd.to_datetime(price_data['Date'])
    events_data['Start'] = pd.to_datetime(events_data['Start'])
    events_data['End'] = pd.to_datetime(events_data['End'])

    # Create a DataFrame that contains each price date with corresponding events
    events_expanded = pd.DataFrame({
        'Date': pd.date_range(start=events_data['Start'].min(), end=events_data['End'].max(), freq='D')
    })

    # Merge the expanded events data with events_data to assign event names
    merged_data = pd.merge_asof(
        events_expanded.sort_values('Date'), 
        events_data.sort_values('Start'), 
        left_on='Date', 
        right_on='Start', 
        direction='backward'
    )

    # Merge the price data with the merged event data
    price_data = price_data.merge(merged_data[['Date', 'Event']], on='Date', how='left')

    # Fill the 'Event' column with 'No Event' where there is no match
    price_data['Event'].fillna('No Event', inplace=True)

    return price_data


# Apply the function to merge events to each price date
price_data = merge_event(events_data, price_data)

/tmp/ipykernel_35802/3594822800.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  price_data['Event'].fillna('No Event', inplace=True)
